In [11]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib widget
plt.close('all')
import numpy as np
from sklearn.preprocessing import StandardScaler
from random import choice
import imageio
import os
import cv2
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

path_save = r"C:\Users\coren\Dropbox\Apps\Snellius2\data_tables\time_plate_infos_agg.pick"

time_plate_info = pd.read_pickle(path_save)
plt.style.use('presentation.mplstyle')

In [2]:
import matplotlib as mpl
cmap1 = mpl.cm.get_cmap('spring')
cmap2 = mpl.cm.get_cmap('winter')

In [25]:
fig, ax = plt.subplots()
table = table.loc[table['out_study']==False]
ax.plot(table['time_since_begin'],table['tot_length_study'])
ax.set_yscale("log")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
def S(t,lamb,C,t0):
    return(C*(1/(1+np.exp(lamb*(t-t0)))))

In [13]:
plt.close('all')
plate = 737
fig, ax = plt.subplots()

ax2 = ax.twinx()
maxi = 12
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
table= table.set_index('t')
max_area = 40
ts = []
ys = []
ys2 = []
for index in range(1,maxi):
    column = f"ring_density_incr-100_index-{index}"
#     column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[column]>=4000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        area = np.sqrt(table["area_sep_comp"][0]+100*index)

        ax.scatter(table[f'time_since_begin_{index}'],table[column],alpha=0.5,color=cmap2(area/max_area))
#         ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.5,color=cmap1(area/max_area))
        popt0, pcov = curve_fit(S, table[f'time_since_begin_{index}'],table[column] ,bounds = ([0,0,-np.inf],3*[np.inf]),p0=[1,1,0])
        ax.plot(table[f'time_since_begin_{index}'],S(table[f'time_since_begin_{index}']))
        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].to_list()
#         ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys))).transpose(),columns=('ts','ys'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-60,60))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()
plt.savefig("Figure/dynamic_single_point_agg.svg")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\coren\anaconda3\envs\cleanMsc\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
popt0

array([ 20.87246335, 518.50178424,  13.22544922])

In [55]:
plt.close('all')
plate = 152
fig, ax = plt.subplots()

ax2 = ax.twinx()
maxi = 12
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    table= table.set_index('t')
    
    ts = []
    ys = []
    ys2 = []
    for index in range(1,maxi):
        column = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            area = np.sqrt(table["area_sep_comp"][0]+100*index)
            
            ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.5,color=cmap2(area/max_area))
            ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.5,color=cmap1(area/max_area))

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].to_list()
            ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
# table= table.set_index('t')
table["area_sep_comp"][99]

2434.240976262188

In [33]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(20):
    column = f"ring_density_incr-100_index-{index}"
    column2 = f"ring_bas_density_incr-100_index-{index}"
    start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax2.set_ylim((-25,650))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('bas density ($\mu m.mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [82]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(20):
    column = f"ring_branch_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].astype(float).to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((-0.00030,0.010))
ax.set_ylabel('new branch density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(20):
    column = f"ring_anas_density_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].astype(float).to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((0,0.0020))
ax2.set_ylim((0,0.25))

ax.set_ylabel('anastomosis density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')
plate = 94
table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
fig, ax = plt.subplots()
ts = []
ys = []
ys2 = []
ax2 = ax.twinx()

for index in range(15):
    column = f"mean_speed_incr-100_index-{index}"
    column2 = f"ring_active_tips_density_incr-100_index-{index}"
    start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
    if not np.isnan(start):
        table[f'time_since_begin_{index}']=table['time_since_begin']-start
        table = table.loc[(table[column_speed]<=400) | (table[column_speed].isnull()==True)]
        
        ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
        ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

        ts +=table[f'time_since_begin_{index}'].to_list()
        ys += table[column].astype(float).to_list()
        ys2+=table[column2].astype(float).to_list()
df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
factor = 4
df['ts_round'] = (df['ts']/factor).astype(int)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,color="blue",linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
ax2.plot(meancurve.index,meancurve2,color="red",linewidth = 5)
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((100,350))
ax.set_ylabel('new branch density ($mm^{-2}.h{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'column_speed' is not defined

In [64]:
plt.close('all')
fig, ax = plt.subplots()

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []


    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
    ax.plot(meancurve.index,meancurve,label=plate)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,label=plate,linestyle="dashed")
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('network density ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.legend()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [65]:
plt.close('all')
fig, ax = plt.subplots()

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []


    for index in range(15):
        column3 = f"ring_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        column = f"mean_speed_incr-100_index-{index}"
        
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            table = table.loc[(table[column_speed]<=400) | (table[column_speed].isnull()==True)]
            
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
    ax.plot(meancurve.index,meancurve,label=plate)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,label=plate,linestyle="dashed")
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((100,350))

ax.set_ylabel('mean speed ($\mu m.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('active tips density ($mm^{-2}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'column_speed' is not defined

In [62]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"mean_speed_incr-100_index-{index}"

        align_col = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[align_col]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            table = table.loc[(table[column_speed]<=400) | (table[column_speed].isnull()==True)]
            
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 4
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
grad = np.gradient(meancurve,meancurve.index)

meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()
ax2.plot(meancurve3.index,meancurve3,linewidth = 5,color='red')

ax.plot(meancurve2.index,meancurve2*meancurve3,linewidth = 5)
ax2.set_ylim((140,250))

# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('n ($mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel('v ($\mu m.h^{-1}$')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'column_speed' is not defined

In [100]:
# plt.close('all')
# fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
# ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column2 = f"mean_speed_incr-100_index-{index}"
        column = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
            table = table.loc[(table[column2]<=400) | (table[column2].isnull()==True)]
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            tstot +=table[f'time_since_begin_{index}'].to_list()
            ystot += table[column].astype(float).to_list()
            ys2tot+=table[column2].astype(float).to_list()
            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
#     ax.plot(meancurve.index,meancurve,color='blue',alpha=0.5)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax2.plot(meancurve.index,meancurve2,color='red',alpha = 0.5)
# df = pd.DataFrame((np.array((tstot,ystot,ys2tot))).transpose(),columns=('ts','ys','ys2'))
# factor = 4
# df['ts_round'] = (df['ts']/factor).astype(int)*factor
# meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,linewidth = 5,color='red')
# # ax2.set_color('red')
# ax.set_xlim((-30,30))
# ax2.set_ylabel('mean speed ($\mu m.h^{-1}$)')
# ax.set_xlabel('shifted time ($h$)')
# ax.set_ylabel('active tips density ($mm^{-2}$)')
# ax.tick_params(axis='y', colors='blue')
# ax2.tick_params(axis='y', colors='red')
# ax2.set_ylim((100,350))

# plt.legend()

In [107]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"mean_speed_incr-100_index-{index}"

        align_col = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[align_col]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
grad = np.gradient(meancurve,meancurve.index)

ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2,color='red',label = 'data')
ax2.plot(meancurve2.index,grad/700,color='red',label = r'$\frac{\partial \rho_{data}}{\partial t}$ (scaled)',linestyle="dashed")

# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel(r'$\rho$ ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$n$ ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
ax2.set_ylim((0,0.15))
plt.legend(fontsize=15)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [114]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"mean_speed_incr-100_index-{index}"

        align_col = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[align_col]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
grad = np.gradient(meancurve,meancurve.index)

ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2*meancurve3*2.56,color='red',label = 'data')
ax2.plot(meancurve2.index,grad,color='red',label = r'$\frac{\partial \rho_{data}}{\partial t}$',linestyle="dashed")

# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel(r'$\rho$ ($\mu m.mm^{-2}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$nv$ ($\mu m.mm^{-2}.h^{-1}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
ax2.set_ylim((0,80))
plt.legend(fontsize=15)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [170]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_anas_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2*meancurve3,linewidth = 5,color='red')
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((0,0.0019))

ax.set_ylabel('anastomosis density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$\rho n$ ($\mu m. mm^{-4}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [150]:
np.max(meancurve)/np.max(meancurve2*meancurve3)

2.1900858932606623e-05

In [152]:
# plt.close('all')
# fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
# ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_anas_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            tstot +=table[f'time_since_begin_{index}'].to_list()
            ystot += table[column].astype(float).to_list()
            ys2tot+=table[column2].astype(float).to_list()
            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
#     ax.plot(meancurve.index,meancurve,color='blue',alpha=0.5)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax.plot(meancurve.index,meancurve,color='blue',alpha = 0.5)
# df = pd.DataFrame((np.array((tstot,ystot,ys2tot))).transpose(),columns=('ts','ys','ys2'))
# factor = 4
# df['ts_round'] = (df['ts']/factor).astype(int)*factor
# meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,linewidth = 5,color='red')
# # ax2.set_color('red')
# ax.set_xlim((-30,30))
# ax.set_ylabel('anastomosis density ($mm^{-2}.h{-1}$)')
# ax.set_xlabel('shifted time ($h$)')
# ax2.set_ylabel('active tips density ($mm^{-2}$)')
# ax.tick_params(axis='y', colors='blue')
# ax2.tick_params(axis='y', colors='red')
# plt.legend()

In [172]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_branch_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2,linewidth = 5,color='red')
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylim((0,0.0074))

ax.set_ylabel('branch density ($mm^{-2}.h^{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$n$ ($mm^{-2}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [175]:
np.max(meancurve)/np.max(meancurve2)

0.05647099251989463

In [176]:
np.max(meancurve)/np.max(meancurve2)/(2.2e-5)

2566.863296358847

In [168]:
# plt.close('all')
# fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
# ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(15):
        column = f"ring_branch_density_incr-100_index-{index}"
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        start = np.min(table.loc[table[f"ring_density_incr-100_index-{index}"]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')

            tstot +=table[f'time_since_begin_{index}'].to_list()
            ystot += table[column].astype(float).to_list()
            ys2tot+=table[column2].astype(float).to_list()
            ts +=table[f'time_since_begin_{index}'].to_list()
            ys += table[column].astype(float).to_list()
            ys2+=table[column2].astype(float).to_list()
    df = pd.DataFrame((np.array((ts,ys,ys2))).transpose(),columns=('ts','ys','ys2'))
    factor = 4
    df['ts_round'] = (df['ts']/factor).astype(int)*factor
    meancurve = df.groupby('ts_round')['ys'].mean()
#     ax.plot(meancurve.index,meancurve,color='blue',alpha=0.5)
    meancurve2 = df.groupby('ts_round')['ys2'].mean()
    ax.plot(meancurve2.index,meancurve,color='blue',alpha = 0.5)
# df = pd.DataFrame((np.array((tstot,ystot,ys2tot))).transpose(),columns=('ts','ys','ys2'))
# factor = 4
# df['ts_round'] = (df['ts']/factor).astype(int)*factor
# meancurve = df.groupby('ts_round')['ys'].mean()
# ax.plot(meancurve.index,meancurve,linewidth = 5)
# meancurve2 = df.groupby('ts_round')['ys2'].mean()
# ax2.plot(meancurve.index,meancurve2,linewidth = 5,color='red')
# # ax2.set_color('red')
# ax.set_xlim((-30,30))
# ax.set_ylabel('anastomosis density ($mm^{-2}.h{-1}$)')
# ax.set_xlabel('shifted time ($h$)')
# ax2.set_ylabel('active tips density ($mm^{-2}$)')
# ax.tick_params(axis='y', colors='blue')
# ax2.tick_params(axis='y', colors='red')
# plt.legend()

In [18]:
plt.close('all')
fig, ax = plt.subplots()
tstot = []
ystot = []
ys2tot = []
ys3tot = []

ax2 = ax.twinx()
for plate in [94,152,792,799,800,91]:
    table = time_plate_info.loc[time_plate_info["Plate"]==plate].copy()
    ts = []
    ys = []
    ys2 = []
    for index in range(20):
        column = f"ring_anas_density_incr-100_index-{index}"
#         column = f"ring_active_tips_density_incr-100_index-{index}"
        
        column2 = f"ring_active_tips_density_incr-100_index-{index}"
        column3 = f"ring_density_incr-100_index-{index}"
        start = np.min(table.loc[table[column3]>=1000]['time_since_begin'])
        if not np.isnan(start):
            table[f'time_since_begin_{index}']=table['time_since_begin']-start
#             ax.plot(table[f'time_since_begin_{index}'],table[column],alpha=0.2,color='blue')
#             ax2.plot(table[f'time_since_begin_{index}'],table[column2],alpha=0.2,color='red')
            
            x = f'time_since_begin_{index}'
            deb = np.min(table[x])
            end = np.max(table[x])
            time = np.linspace(deb,end,1000)
            tstot +=list(time)
            f = interp1d(table[x], table[column])
            ystot += list(f(time))
            f = interp1d(table[x], table[column2])
            ys2tot+=list(f(time))
            f = interp1d(table[x], table[column3])
            ys3tot+=list(f(time))
df = pd.DataFrame((np.array((tstot,ystot,ys2tot,ys3tot))).transpose(),columns=('ts','ys','ys2','ys3'))
factor = 1
df['ts_round'] = np.floor(df['ts']/factor)*factor
meancurve = df.groupby('ts_round')['ys'].mean()
ax.plot(meancurve.index,meancurve,linewidth = 5)
meancurve2 = df.groupby('ts_round')['ys2'].mean()
meancurve3 = df.groupby('ts_round')['ys3'].mean()

ax2.plot(meancurve2.index,meancurve2*meancurve3,linewidth = 5,color='red')
# ax2.set_color('red')
ax.set_xlim((-30,30))
ax.set_ylabel('anastomosis density ($mm^{-2}.h{-1}$)')
ax.set_xlabel('shifted time ($h$)')
ax2.set_ylabel(r'$\rho n$ ($\mu m. mm^{-}$)')
ax.tick_params(axis='y', colors='blue')
ax2.tick_params(axis='y', colors='red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …